<span type="title">内部类</span> | <span type="update">2018-07-23</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要讲解内部类。</p></span>

# 内部类的定义和使用

内部类是一种位于类内部的类。其允许将逻辑相关的代码放在一个“块”中，因此可以提高程序的清晰性。内部类不是“组合”，前者提供了一些类方法的实现，而“组合”则重在实例化现有的类，尽管从表面看来，它们在一个类中都持有其他的类。

内部类常常被用来**隐藏代码**，隐藏对于一个接口的具体实现过程。其可以和外围类进行通信，随意调用外围类的私有域和方法，即便声称了 `private`，但是外围类只有在实例化内部类之后才能访问内部类的动态域和方法。

内部类常常被用作一种创建**紧密但又清晰的类和类之间的关系**的一种方式。而在内部类之前，根据我们的介绍，类和类想要建立联系，必须通过 `相同的接口实现` 或者 `驴头不对马嘴的继承`。而通过内部类，我们可以将几个保存不同状态的类放置到一个类的内部，然后通过手动控制需要选用的类来改变这个外围类的状态。这常常在GUI或者控制器设计模式中较为常见。正因为此，我们才实现了真正的可以媲美C++的，更加优雅的基于类而不是接口的正经的多重继承。

```java
/Sequence.java
import java.util.Random; import static com.mazhangjing.Print.*;
/**
 * 选择器，包括三个方法 end() 是否结束；current()当前对象；next() 计数器加1，不返回值 */
interface Selector {
    boolean end();
    Object current();
    void next();
}
/**
 * 一个定常的序列类，数据保存在 sequence 数组中，next 表示当前序列长度
 * 私有类 sequenceselecter 实现了 selector，在内部操纵计数器 i，完成了判断是否结束、取回当前序列
 * 对应位置对象和计数器加1的操作。
 * */
public class Sequence {
    private Object[] sequence;
    private int next = 0;

    public Sequence(int size){
        sequence = new Object[size];
    }

    public void add(Object n){
        if (next < sequence.length){
            sequence[next++] = n; }
    }
    /**
     * 内部的 SenquenceSelecter 类可以自动获取外部类所有成员权限，不论是否私有。
     * 外部类需要一个方法来传递构造器构造好的内部类对象，在这里使用外部对象 getSelector()，或者
     * 使用 x.new SequenceSelecter() 创建，注意，不能写成 new Sequence.SequenceSelecter();
     * 外部类总是先于内部类进行对象的创建（其中的愿意那时内部类总要引用其所需要的
     * 外部类域或者方法）*/
    private class SequenceSelecter implements Selector {
        private int i = 0;
        public boolean end() {
            return i == sequence.length; }
        public Object current() {
            return sequence[i]; }
        public void next() {
            if (i < sequence.length) {i++;} }
        //对外部类的引用，返回值写外部类名，具体调用写类名.this，返回的是外部对象。
        public Sequence getFather(){
            return Sequence.this;
        }
    }
    //用以说明内部类的接口比外部类更加灵活的例子——可以对相同接口不同的实现
    private class reverseSelecter implements Selector {
        private int i = sequence.length;
        public boolean end() {
            return i == 0; }
        public Object current() {
            return sequence[i-1]; }
        public void next() {
            if (i > 0) {i--;} }
    }

    public Selector getSelector(){ return new SequenceSelecter();}

    public static void main(String[] args){
        Sequence x = new Sequence(10);
        CM2 a = new CM2("Hello World");
        CM2 b = new CM2("Hello CORKINE");
        for (int i = 0; i < 8; i++){ x.add(Integer.toString(i)); }
        x.add(a);
        x.add(b);
        //Selector s = x.getSelector(); 使用外部类方法创建内部类实例
        //需要注意，这里因为是一个接口，所以写成了Selector，如果不想向上转型，默认写作
        //Sequence.SequenceSelecter s = x.getSelector();
        //或者写成下面这样，使用.new直接创建内部类实例
        //因为外部类必须先存在，因此使用 x.new 而不是 new Sequence.SequenceSelecter()
        Sequence.SequenceSelecter s = x.new SequenceSelecter();
        Selector s2 = x.new reverseSelecter();
        while (!s2.end()){
            print(s2.current());
            s2.next();
        }
    }
}
class CM2 {
    private String s;
    CM2(String s){ this.s = s; }
    public String toString() { return this.s; }
}
```

在这个例子中，内部类 `SequenctSelector` 是对于接口 `Selector` 的一种实现。而和之前不同，没有使用外围类 `Sequence` 自身 `implement` 这个接口的原因是，其自身的实现很蹩脚，一个序列实现一个计数控制器从逻辑上来讲很别扭。但是在内部类之前，我们只有这一种选择。

**内部类的语法和细节**

注意这里的内部类语法，